In [1]:
%pylab tk

import numpy as np
import os
import json
from time import time
from skimage import io, color
from skimage.filter.rank import median, mean
from skimage.transform import rescale
from skimage.morphology import disk
from skimage.measure import label, regionprops
import cv2

BASE_PATH = os.getcwd()
print("Current base path: {0}".format(BASE_PATH))
DATA_PATH = BASE_PATH + '/Daten/2D/Talus_dorsal/'
TO_PATH = BASE_PATH + '/Daten/2D/Talus_dorsal_cut_2/'
TEST_FILES = json.loads(open(BASE_PATH + '/Daten/2D/testfiles.json').read())

MU = 1.3
SCALE = 0.1
INITIAL_WINDOW_SIZE = 500
BORDER_SIZE = 100
PADDING = 100

def calculate_possible_bone_pixels(hsv_image, certain_bone_pixels, certain_non_bone_pixels):
    #smoothed_hsv = mean(hsv_image[:, :, 0], disk(50))
    smoothed_hsv = cv2.blur(hsv_image[:, :, 0], (51, 51))
    threshold = MU * np.mean(smoothed_hsv[certain_bone_pixels])
    
    possible_bones = np.zeros_like(hsv_image[:, :, 0], np.uint)
    possible_bones[smoothed_hsv < threshold] = 1
    possible_bones[certain_non_bone_pixels] = 0
    
    return possible_bones

def calculate_certain_bone_pixels(hsv_image):
    low_res_image = rescale(hsv_image[:, :, 0], SCALE)
    local_mean = mean(low_res_image, disk(50))
    local_minimum_flat = np.argmin(local_mean)
    local_minimum = np.multiply(np.unravel_index(local_minimum_flat, low_res_image.shape), round(1 / SCALE))
    
    certain_bone_pixels = np.full_like(hsv_image[:, :, 0], False, bool)
    certain_bone_pixels[
        local_minimum[0]-INITIAL_WINDOW_SIZE/2:local_minimum[0]+INITIAL_WINDOW_SIZE/2,
        local_minimum[1]-INITIAL_WINDOW_SIZE/2:local_minimum[1]+INITIAL_WINDOW_SIZE/2
    ] = True
    
    return certain_bone_pixels

def calculate_certain_non_bone_pixels(hsv_image):
    certain_non_bone_pixels = np.full_like(hsv_image[:, :, 0], False, bool)
    certain_non_bone_pixels[0:BORDER_SIZE,:] = True
    certain_non_bone_pixels[-BORDER_SIZE:-1,:] = True
    certain_non_bone_pixels[:,0:BORDER_SIZE] = True
    certain_non_bone_pixels[:,-BORDER_SIZE:-1] = True
    
    return certain_non_bone_pixels

def find_largest_label(possible_bone_pixels):
    #labels = cv2.connectedComponents(possible_bone_pixels, 4)
    labels = label(possible_bone_pixels, neighbors=4, background=0)
    labels = labels + 1
    
    label_props = regionprops(labels)
    label_with_max_area = label_props[0]
    for labelprop in label_props:
        if label_with_max_area.area <= labelprop.area:
            label_with_max_area = labelprop
    return labels == label_with_max_area.label

def cut_bone_from_full_image(rgb, bone_pixels):
    indices_of_bone_pixels = np.where(bone_pixels == True)
    min_dim_0 = indices_of_bone_pixels[0].min() - PADDING
    max_dim_0 = indices_of_bone_pixels[0].max() + PADDING
    min_dim_1 = indices_of_bone_pixels[1].min() - PADDING
    max_dim_1 = indices_of_bone_pixels[1].max() + PADDING
    
    min_dim_0 = max(min_dim_0, 0)
    min_dim_1 = max(min_dim_1, 0)
    max_dim_0 = min(max_dim_0, rgb.shape[0])
    max_dim_1 = min(max_dim_1, rgb.shape[1])
    
    return np.copy(rgb[min_dim_0:max_dim_0,min_dim_1:max_dim_1,:])

Populating the interactive namespace from numpy and matplotlib
Current base path: /media/lau/FLASH DRIVE


In [3]:
file = TEST_FILES[3]
ascii_filename = file['filename'].encode('ascii', 'replace')

rgb_image = io.imread(DATA_PATH + file['filename'])
hsv_image = color.rgb2hsv(rgb_image)

start_time = time()
certain_bone_pixels = calculate_certain_bone_pixels(hsv_image)
print('{0} calculate_certain_bone_pixels: {1}s'.format(ascii_filename, time() - start_time))

start_time = time()
certain_non_bone_pixels = calculate_certain_non_bone_pixels(hsv_image)
print('{0} calculate_certain_bone_pixels: {1}s'.format(ascii_filename, time() - start_time))

start_time = time()
possible_bone_pixels = calculate_possible_bone_pixels(hsv_image, certain_bone_pixels, certain_non_bone_pixels)
print('{0} calculate_possible_bone_pixels: {1}s'.format(ascii_filename, time() - start_time))

start_time = time()
bone_pixels = find_largest_label(possible_bone_pixels)
print('{0} find_label: {1}s'.format(ascii_filename, time() - start_time))

start_time = time()
bone_only = cut_bone_from_full_image(rgb_image, bone_pixels)
print('{0} cut_bone_from_full_image: {1}s'.format(ascii_filename, time() - start_time))

watershed_config = np.copy(rgb_image)
watershed_config[:,:,0][certain_bone_pixels] = 255
watershed_config[:,:,1][certain_non_bone_pixels] = 255

watershed_result = np.copy(rgb_image)
watershed_result[:,:,0][bone_pixels] = 255

fig, ax = plt.subplots(1, 5)

ax[0].imshow(rgb_image)
ax[0].set_title("{0} RGB".format(file['description']))
ax[1].imshow(color.gray2rgb(possible_bone_pixels))
ax[1].set_title("Possible Bone")
ax[2].imshow(watershed_config)
ax[2].set_title("Config")
ax[3].imshow(watershed_result)
ax[3].set_title("Result")
ax[4].imshow(bone_only)
ax[4].set_title("Bone")

draw()

G?_3_2000_3_L.JPG calculate_certain_bone_pixels: 0.0674171447754s
G?_3_2000_3_L.JPG calculate_certain_bone_pixels: 0.00132894515991s
G?_3_2000_3_L.JPG calculate_possible_bone_pixels: 0.0684659481049s
G?_3_2000_3_L.JPG find_label: 1.01443099976s
G?_3_2000_3_L.JPG cut_bone_from_full_image: 0.0302810668945s


In [2]:
from glob import glob
from os import path

all_files = glob(DATA_PATH + '*')

for index, filename in enumerate(all_files):
    basename = path.basename(filename)
    
    print("{0} - {1} of {2}".format(basename, index+1, len(all_files)))
    
    rgb_image = io.imread(filename)
    hsv_image = color.rgb2hsv(rgb_image)
    
    certain_bone_pixels = calculate_certain_bone_pixels(hsv_image)
    certain_non_bone_pixels = calculate_certain_non_bone_pixels(hsv_image)
    possible_bone_pixels = calculate_possible_bone_pixels(hsv_image, certain_bone_pixels, certain_non_bone_pixels)
    bone_pixels = find_largest_label(possible_bone_pixels)
    bone_only = cut_bone_from_full_image(rgb_image, bone_pixels)
    
    io.imsave(TO_PATH + basename, bone_only)

GSR2013-96R.JPG - 1 of 112
MiZ08-465R_8.JPG - 2 of 112
MiZ09-113R_1.JPG - 3 of 112
GÜ2011_GÜ17_R.JPG - 4 of 112
GSR2013-100R.JPG - 5 of 112
Ma338.5L_d.JPG - 6 of 112
GSR2013-85R.JPG - 7 of 112
GÜ2011_GÜ14_R.JPG - 8 of 112
MiZ08-465L_1.JPG - 9 of 112
GÜ001_l3_L.JPG - 10 of 112
GSR2014_37_L.JPG - 11 of 112
MiZ08-465L_3.JPG - 12 of 112
MiZ08-465R_5.JPG - 13 of 112
GÖ_p_2013_188L.JPG - 14 of 112
Gaz_3.JPG - 15 of 112
GÖ_3_2008_9_L.JPG - 16 of 112
MiZ09-118R_3.JPG - 17 of 112
Ma61.1L_d.JPG - 18 of 112
GÖ_3_2012_421_R.JPG - 19 of 112
MiZ08-465L_15.JPG - 20 of 112
GÖ_3_2011_281_R.JPG - 21 of 112
GÖ_3_2000_2_R.JPG - 22 of 112
GÖ_p_2012_188_L.JPG - 23 of 112
GÖ_i_2012_171_R.JPG - 24 of 112
GSR2013-93L.JPG - 25 of 112
Ma300.4L_d.JPG - 26 of 112
GÖ_3_2011_297_L.JPG - 27 of 112
GSR2013-23R.JPG - 28 of 112
GÜ2011_GÜ12_R.JPG - 29 of 112
Gaz_2.JPG - 30 of 112
GÖ_p_2010_120_L.JPG - 31 of 112
Ma354R_d.jpg - 32 of 112
MiZ08-465R_6.JPG - 33 of 112
GÖ_3_L9_66_87.1_R.JPG - 34 of 112
GSR2013-22R.JPG - 35 of

/home/l/lau/work/master/venv/local/lib/python2.7/site-packages/skimage/util/dtype.py:107: UserWarning: Possible precision loss when converting from float64 to uint8
  "%s to %s" % (dtypeobj_in, dtypeobj))
